# 爬虫示例

接下来我们给三个示例，来讲解浏览器抓包以及headers设置（知乎），mangoDB（拉钩），还有selenium爬取动态网页（淘宝。）

# 爬虫思路解析：

- 抓取网页，分析请求（先查看robots.txt，对于个人抓取虽然遵守与否影响不大，但是里面有时候给你很多有用的信息，所以还是参考一下比较好）
- 解析网页，寻找数据
- 储存数据，多页处理


## 翻页问题如何处理--知乎用户信息

翻页后url不变应该如何处理？有哪些网站是这样的呢？

首先打开对应网址，打开控制台并点击到Network标签。此时刷新网页，分析每一个请求，多点一点各个标签（headers，preview等，检查所有的信息）

In [ ]:
# coding: utf-8

import requests
import pandas as pd

headers = {'authorization':'Bearer 2|1:0|10:1509084229|4:z_c0|80:MS4xQWRvM0FBQUFBQUFtQUFBQVlBSlZUVVVhNEZvY1RZYm5rcTU2ckotMjlOdVkyX0ZBRTh6alNRPT0=|c2988401466531b66af52d04bb009a5e95942c4ef13bfc2aec404e804a619406',
          'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36',
          }

user_data = []
def get_user_data(page_num):
    for i in range(page_num):
        
        url = "https://www.zhihu.com/api/v4/members/ljyduke/followees?include=data%5B*%5D.answer_count%2Carticles_count%2Cgender%2Cfollower_count%2Cis_followed%2Cis_following%2Cbadge%5B%3F(type%3Dbest_answerer)%5D.topics&offset={}&limit=20".format(i*20) # 这里关于offset做一个简单的解释，一般来说它表征的是从基准点（这里是首页）开始往后‘离开’多少。limit是限制当前页面的信息个数
        response = requests.get(url,headers=headers)
        data = response.json()['data'] 
        user_data.extend(data)# 在列表的末尾一次性追加多个值，这个和append不同，可以自行实验
'''
json
对象表示为键值对
数据由逗号分隔
花括号保存对象
方括号保存数组
'''

In [ ]:
get_user_data(3)
df = pd.DataFrame.from_dict(user_data)
print(df.head)
print(response.status_code)# 打印出请求状态码，200 即为正常，别的码就是错误，可以上网去对应的寻找。其中403就是代表你的ip被网站封了，因为你作为一个小爬虫被发现啦哈哈哈
df.to_csv('user.csv')# 可以保存在本地

这里我们看到代码返回了一个500 server error。关于各个错误代码，请参照这个link：
http://tool.oschina.net/commons?type=5

但是在这里，是因为我们爬虫的时候所发送的请求无法被对方所理解，所以我们需要伪装成浏览器来发送请求。即添加user agent（UA）。

## 拉勾网职位信息

我没有在代码中加数据库 关于mongoDB 安装：

https://docs.mongodb.com/manual/tutorial/install-mongodb-on-os-x/?_ga=2.93535005.1297227622.1510894879-29904962.1510894879


In [ ]:
import requests

url = 'https://www.lagou.com/jobs/positionAjax.json?city=%E5%8C%97%E4%BA%AC&needAddtionalResult=false&isSchoolJob=0'

payload = {
    'first':'true',
    'pn':'1',
    'kd':'爬虫'
}
# 这就是我们的请求中所带着的内容，即我们和服务器请求的内容
headers = {
    'Cookie':'user_trace_token=20171109151642-c4222f19-ce5a-414f-8d08-570ba177c078; LGUID=20171109151644-f0e40786-c51d-11e7-82d0-525400f775ce; JSESSIONID=ABAAABAAAGGABCB71B7FC8FA2DBC084397747F3138B7118; _gat=1; PRE_UTM=; PRE_HOST=; PRE_SITE=; PRE_LAND=https%3A%2F%2Fwww.lagou.com%2Fjobs%2Flist_%25E6%25B7%25B1%25E5%25BA%25A6%25E5%25AD%25A6%25E4%25B9%25A0%3Fpx%3Ddefault%26city%3D%25E5%258C%2597%25E4%25BA%25AC; _gid=GA1.2.374477419.1510920136; Hm_lvt_4233e74dff0ae5bd0a3d81c6ccf756e6=1510211805,1510920137; Hm_lpvt_4233e74dff0ae5bd0a3d81c6ccf756e6=1510920147; _ga=GA1.2.1403089702.1510211805; LGSID=20171117200216-27d54642-cb8f-11e7-95cc-525400f775ce; LGRID=20171117200227-2e31788d-cb8f-11e7-95cc-525400f775ce; SEARCH_ID=326cb0fe98e3457a81659fbff07edd97',
    'Referer':'https://www.lagou.com/jobs/list_%E7%88%AC%E8%99%AB?city=%E5%8C%97%E4%BA%AC&cl=false&fromSearch=true&labelWords=&suginput=',
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36'
}
# headers 就是在请求的时候会使用的包头，这些包头内容都会被服务器读取到，所以我们可以利用这个来伪装自己变成一个“浏览器”
# 所以我们可以在网站中寻找一些headers来伪装自己就不会被认出来啦
response = requests.post(url,data = payload,headers = headers)

# print(response.text)
print(response.json()['content'])

# 关于这里你会发现点击下一页url是不会改变的。所以可以探究一下怎么才可以翻页爬取呢？（TIPS：pn ）

# 对于不同的网页中，你会发现他的翻页的url构造并不相同，所以可以根据具体的情景来编写自己的爬虫

In [ ]:
# 这里是一个优化后的爬虫代码，使用到了 fake_useragent 我并没有配置数据库，看需求吧
import time
from fake_useragent import UserAgent
import requests



# 这就是我们的请求中所带着的内容，即我们和服务器请求的内容
headers = {
    'Cookie':'user_trace_token=20171109151642-c4222f19-ce5a-414f-8d08-570ba177c078; LGUID=20171109151644-f0e40786-c51d-11e7-82d0-525400f775ce; JSESSIONID=ABAAABAAAGGABCB71B7FC8FA2DBC084397747F3138B7118; _gat=1; PRE_UTM=; PRE_HOST=; PRE_SITE=; PRE_LAND=https%3A%2F%2Fwww.lagou.com%2Fjobs%2Flist_%25E6%25B7%25B1%25E5%25BA%25A6%25E5%25AD%25A6%25E4%25B9%25A0%3Fpx%3Ddefault%26city%3D%25E5%258C%2597%25E4%25BA%25AC; _gid=GA1.2.374477419.1510920136; Hm_lvt_4233e74dff0ae5bd0a3d81c6ccf756e6=1510211805,1510920137; Hm_lpvt_4233e74dff0ae5bd0a3d81c6ccf756e6=1510920147; _ga=GA1.2.1403089702.1510211805; LGSID=20171117200216-27d54642-cb8f-11e7-95cc-525400f775ce; LGRID=20171117200227-2e31788d-cb8f-11e7-95cc-525400f775ce; SEARCH_ID=326cb0fe98e3457a81659fbff07edd97',
    'Referer':'https://www.lagou.com/jobs/list_%E7%88%AC%E8%99%AB?city=%E5%8C%97%E4%BA%AC&cl=false&fromSearch=true&labelWords=&suginput=',
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36'
}
# headers 就是在请求的时候会使用的包头，这些包头内容都会被服务器读取到，所以我们可以利用这个来伪装自己变成一个“浏览器”
# 所以我们可以在网站中寻找一些headers来伪装自己就不会被认出来啦

def get_job_info(num_page, kw):
    for i in range(num_page):
        
        url = 'https://www.lagou.com/jobs/positionAjax.json?city=%E5%8C%97%E4%BA%AC&needAddtionalResult=false&isSchoolJob=0'

        payload = {
            'first':'true',
            'pn': str(i),
            'kd':kw
        }
        ua = UserAgent()
        header['User-Agent'] = ua.random # fake_useragent 中的 UserAgent 的一个方法，可以随机选择不同浏览器
        response = requests.post(url,data = payload,headers = headers)
        # print(response.text)
        if response.status_code == 200: # 这里是检查访问状态是否是对的如果是403就代表被封了。。。
            job_json = response.json(['content']['positionResult']['result'])
            print(job_json)
            
        else:
            print('something wrong')

# 关于这里你会发现点击下一页url是不会改变的。所以可以探究一下怎么才可以翻页爬取呢？（TIPS：pn ）

# 对于不同的网页中，你会发现他的翻页的url构造并不相同，所以可以根据具体的情景来编写自己的爬虫

In [ ]:
get_job_info(3,'Python')

## 这里将会爬取淘宝的数据 使用 Selenium  

大胆的去跑跑跑～～～

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# 但是这个库 selenium 的使用 所需要的时间以及别的开销例如需要装driver of browser 太费功夫了
# 不过使用它就不用管headers的问题啦，因为我们已经模拟了浏览器的功能
driver = webdriver.Chrome() # 使用webdrive的chrome的方法，打开chrome浏览器
driver.get('http://www.baidu.com') # 使用chrome浏览器搜索百度这个网站

elem = driver.find_element_by_xpath("//*[@id='kw']") # 在浏览器的页面source code中找到id为kw的关键字，就是我们的输入框
elem.send_keys("Python selenium",Keys.ENTER) # ，然后送入Python selenium的输入，并且调用key的enter方法，相当于是按下了enter键
print(driver.page_source) # 然后将页面元素打印出来

# 还有一个叫做无头浏览器 PhantomJS()这个的话就不会在前台打来浏览器，只会在后台默默的坐一切。
# driver = webdriver.PhantomJS() # 使用webdrive的chrome的方法，打开chrome浏览器
# driver.get('http://www.baidu.com') # 使用chrome浏览器搜索百度这个网站

# elem = driver.find_element_by_xpath("//[@id='kw']") # 在浏览器的页面source code中找到id为kw的关键字，就是我们的输入框
# elem.send_keys("Python selenium",Keys.ENTER) # ，然后送入Python selenium的输入，并且调用key的enter方法，相当于是按下了enter键
# print(driver.page_source)


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from pyquery import PyQuery as pq 
# http://www.jianshu.com/p/c07f7cd1b548

# from pymongo import MongoClient 我注释掉了mongoDB 
import re


browser = webdriver.Chrome() # 使用chrome的网页驱动
wait = WebDriverWait(browser, 10) # webdriver的等待时间

# client = MongoClient()
# db=client.taobao
# data = db.data

# 这个就是尝试与了解，另外对于一些比较困难的网站可以使用这个方式来解决问题。
def search(kd):
    try:
        browser.get('https://www.taobao.com/')
        input = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#q")))
        submit = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,'#J_TSearchForm > div.search-button > button')))
        input.send_keys(kd)
        submit.click()# 提交一个点击时间
        total = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,'#mainsrp-pager > div > div > div > div.total')))
        get_products()
        return total.text
    except TimeoutException:
        return search()

def next_page(page_number):
    try:
        input = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#mainsrp-pager > div > div > div > div.form > input")))
        submit = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#mainsrp-pager > div > div > div > div.form > span.btn.J_Submit')))
        input.clear()
        input.send_keys(page_number)
        submit.click()
        wait.until(EC.text_to_be_present_in_element((By.CSS_SELECTOR,'#mainsrp-pager > div > div > div > ul > li.item.active > span'),str(page_number)))
        get_products()
    except TimeoutException:
        next_page(page_number)

def get_products():
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,'#mainsrp-itemlist .items .item')))
    html = browser.page_source
    doc = pq(html)
    items = doc('#mainsrp-itemlist .items .item').items()
    for item in items:
        product = {
            'image':item.find('.pic .img').attr('src'),
            'price':item.find('.price').text(),
            'deal':item.find('.deal-cnt').text()[:-3],
            'title':item.find('.title').text(),
            'shop':item.find('.shop').text(),
            'location':item.find('.location').text(),
        }
        print(product)
#         data.insert(product)

def main(kd):
    total = search(kd)
    total = int(re.compile('(\d+)').search(total).group(1))
    for i in range(2, total+1):
        next_page(i)


if __name__ == '__main__':
    main('大衣')
    
    
'''
两种 HTTP 请求方法：GET 和 POST
在客户机和服务器之间进行请求-响应时，两种最常被用到的方法是：GET 和 POST。
GET - 从指定的资源请求数据。
POST - 向指定的资源提交要被处理的数据
'''